<a href="https://colab.research.google.com/github/innovativenexusbd/AgroAI/blob/main/nutrientdataprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [15]:
np.random.seed(42)
data_size = 1000
nutrients = {
    #Should be collected from the IoT sensors
}
# Nutrient sufficiency thresholds. Should be adjusted based on the crops and sources.
sufficiency_thresholds = {
    'Nitrogen': 50,
    'Phosphorus': 30,
    'Potassium': 40,
    'Calcium': 20,
    'Magnesium': 10,
    'Sulfur': 15,
    'Boron': 1,
    'Copper': 0.5,
    'Iron': 2.5,
    'Manganese': 2,
    'Zinc': 1
}
def is_sufficient(row):
    for nutrient, threshold in sufficiency_thresholds.items():
        if row[nutrient] < threshold:
            return 'Insufficient'
    return 'Sufficient'

df = pd.DataFrame(nutrients)
df['Sufficiency'] = df.apply(is_sufficient, axis=1)
df.to_csv('soil_nutrients.csv', index=False)

Load Dataset, Prepare Data and Split the Dataset

In [10]:
df = pd.read_csv('soil_nutrients.csv')
X = df.drop(columns=['Sufficiency'])
y = df['Sufficiency']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Scale the features and Train the Model

In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)


KNeighborsClassifier()

Prediction and ensuring that weather the nutrients are sufficient or insufficient. If insufficient, which nutrients are insufficient

In [12]:
y_pred = knn.predict(X_test_scaled)
print(classification_report(y_test, y_pred))
def check_insufficient_nutrients(row, thresholds):
    insufficient_nutrients = []
    for nutrient, value in row.items():
        if nutrient in thresholds and value < thresholds[nutrient]:
            insufficient_nutrients.append(nutrient)
    return insufficient_nutrients
results = X_test.copy()
results['Predicted_Sufficiency'] = y_pred

insufficient_nutrients_list = []
for i in range(len(X_test)):
    if y_pred[i] == 'Insufficient':
        insufficient_nutrients = check_insufficient_nutrients(X_test.iloc[i], sufficiency_thresholds)
        insufficient_nutrients_list.append(", ".join(insufficient_nutrients))
    else:
        insufficient_nutrients_list.append("")

results['Insufficient_Nutrients'] = insufficient_nutrients_list


              precision    recall  f1-score   support

Insufficient       0.82      0.91      0.87       148
  Sufficient       0.64      0.44      0.52        52

    accuracy                           0.79       200
   macro avg       0.73      0.68      0.69       200
weighted avg       0.78      0.79      0.78       200



Saving the Ultimate Result in the CSV format

In [13]:
results.to_csv('predicted_soil_nutrients.csv', index=False)
